# Acquire Live Event Ticketing Data

In [2]:
import pandas as pd
import json
import datetime
import requests
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import config
%matplotlib inline

## Acquire Ticketing Data from Ticketmaster

In [3]:
#DMA and Market numbers
#dma_list = [382,362,264]
#mkt_list =  [42,27,14,36,40,22,5,3,31,10,11,35,47,15]

# Function to get the number of pages of ticketmaster data
def get_number_of_TM_pages(market,source='ticketmaster,frontgate',startDate=None,endDate=None):
    url = 'https://app.ticketmaster.com/discovery/v2/events.json?countryCode=US'
    payload = {'source': source,
               'classificationName': 'music',
               'size': '200',
               'marketId': market,
               'startDateTime' : startDate,
               'endDateTime' : endDate,
               'apikey': config.tm_key}
    r = requests.get(url,params=payload,verify=True)
    json_obj = json.loads(r.text)
    return json_obj.get('page').get('totalPages')

#Get TicketMaster data, return a dataframe
def getTicketMasterData(pageNumber,market,source='ticketmaster,frontgate',startDate=None,endDate=None):
    url = 'https://app.ticketmaster.com/discovery/v2/events.json?countryCode=US'
    payload = {'source': source,
               'marketId': market,
               'classificationName' : 'music',
               'size': '200',
               'page': pageNumber,
               'startDateTime' : startDate,
               'endDateTime' : endDate,
               'apikey': config.tm_key}
    r = requests.get(url,params=payload,verify=True)
    json_response = json.loads(r.text)
    event_info = []
    for event in json_response.get('_embedded',{}).get('events',{}):
         event_info.append({
            'TM_id': str(event.get('id',{})),
            'TM_name' : str(event.get('name',{})),
            'TM_artist': list(str(attraction.get('name')) for attraction in event['_embedded'].get('attractions',{})),
            'TM_venue' : list(str(venue.get('name')) for venue in event['_embedded'].get('venues',{}))[0],
            'TM_venue_city' : list(str(venue.get('city',{}).get('name')) for venue in event['_embedded'].get('venues',{}))[0],
            'TM_venue_state' : list(str(venue.get('state',{}).get('stateCode')) for venue in event['_embedded'].get('venues',{}))[0],
            'TM_venue _lat' : list(str(venue.get('location',{}).get('latitude')) for venue in event['_embedded'].get('venues',{}))[0],
            'TM_venue_long' :list(str(venue.get('location',{}).get('longitude')) for venue in event['_embedded'].get('venues',{}))[0],
            'TM_date' : str(event.get('dates',{}).get('start',{}).get('dateTime',{})),
            'TM_timezone' : str(event.get('dates',{}).get('timezone')),
            'TM_span_multiple_days' : str(event.get('dates',{}).get('spanMultipleDays')),
            'TM_presale_date_start' : list(str(presale.get('startDateTime',{})) for presale in event.get('sales').get('presales',{})),
            'TM_presale_date_end' : list(str(presale.get('endDateTime',{})) for presale in event.get('sales').get('presales',{})),
            'TM_sale_date_start' : str(event.get('sales',{}).get('public',{}).get('startDateTime')),
            'TM_FV_prices': event.get('priceRanges'),
            'TM_promoter': str(event.get('promoter',{}).get('name')),
            'TM_genre' : event.get('classifications'),
         })
    tmDF = pd.DataFrame(event_info)
    return tmDF

#Convert timedate information from UTC to local time
def convert_times(df,times_list,tz_col):
    #Loop through each timezone
    df_list = []
    for tz in df[tz_col].unique():
        #Filter rows by timezone
        mask = (df[tz_col] == tz)
        df_local = df.loc[mask]
        #Loop through each datetime row
        for col in times_list:
            #Convert each column to datetime series, localize to UTC and then convert to proper timezone
            df_local[col] = pd.to_datetime(df_local[col],errors='coerce').dt.tz_localize('UTC').dt.tz_convert(tz)
            #Convert each column to datetime series, localize to UTC and then convert to proper timezone
        df_list.append(df_local)
    df = pd.concat(df_list, axis=0)
    return df

#Explode out columns with nested information
def explode(df,col,index_col):
    df1 = df[col].apply(pd.Series)
    df1 = df1[0].apply(pd.Series)
    df1.index = df[index_col]
    df = df.merge(df1, how='left', left_on=index_col, right_index=True)
    df = df.drop(col,axis=1)
    return df

In [5]:
#Create list US Ticketmaster market numbers -- Look at TM API docs for market number info.
mkt_list = [40] # 40 is San Antonio and Austin Market

**Ticketmaster API limits the amount of data that can be pulled per day**
- Initial Data Acquisition is returning:
    - Concert events in market 40 (San Antonio & Austin)
    - Occuring between the dates April 1, 2021 and May 1, 2021

In [6]:
tm_df_list = []
p1startDate = '2021-04-01T00:00:00Z'
p1endDate = '2021-05-01T00:00:00Z'

#Create dictionaries of dma/mkt numbers, and number of pages of data
mkt_dict_p1 = dict()
mkt_dict_p2 =dict()

#Get number of pages for face value market items
for mkt in mkt_list:
    mkt_dict_p1[mkt] = get_number_of_TM_pages(market = mkt,
                                              startDate = p1startDate,
                                              endDate = p1endDate)
    mkt_dict_p2[mkt] = get_number_of_TM_pages(market = mkt,
                                              startDate = p1endDate)
    
#Get market data
for mkt in mkt_dict_p1.keys():
    for current_page in range(0,mkt_dict_p1[mkt]):
        tm_df_list.append(getTicketMasterData(market = mkt,
                                              pageNumber = current_page,
                                              startDate = p1startDate,
                                              endDate = p1endDate))
for mkt in mkt_dict_p2.keys():
    for current_page in range(0,mkt_dict_p2[mkt]):
        tm_df_list.append(getTicketMasterData(market = mkt,
                                              pageNumber = current_page,
                                              startDate = p1endDate))

In [7]:
#Combine all dataframes for full data on events and face_value prices
tm_df = pd.concat(tm_df_list,axis=0)

tm_df.head()

,TM_id,TM_name,TM_artist,TM_venue,TM_venue_city,TM_venue_state,TM_venue _lat,TM_venue_long,TM_date,TM_timezone,TM_span_multiple_days,TM_presale_date_start,TM_presale_date_end,TM_sale_date_start,TM_FV_prices,TM_promoter,TM_genre
0,G5dIZp0wKy-SF,Justin Furstenfeld of Blue October (21+ Event)...,[Justin Furstenfeld],Whitewater Amphitheater,New Braunfels,TX,29.862013,-98.156969,2021-04-24T01:00:00Z,America/Chicago,False,[],[],2021-02-19T16:00:00Z,"[{'type': 'standard', 'currency': 'USD', 'min'...",PROMOTED BY VENUE,"[{'primary': True, 'segment': {'id': 'KZFzniwn..."
1,G5dIZpbCwKv7f,Jamestown Revival (21+ Event) (Reduced Capacit...,[Jamestown Revival],Whitewater Amphitheater,New Braunfels,TX,29.862013,-98.156969,2021-04-04T01:00:00Z,America/Chicago,False,[],[],2020-12-11T16:00:00Z,"[{'type': 'standard', 'currency': 'USD', 'min'...",PROMOTED BY VENUE,"[{'primary': True, 'segment': {'id': 'KZFzniwn..."
2,G5dIZ4URqF9t2,The Monkees,[The Monkees],Austin City Limits Live at The Moody Theater,Austin,TX,30.26536691,-97.7471222,2021-04-03T01:00:00Z,America/Chicago,False,[],[],2019-11-15T16:00:00Z,"[{'type': 'standard', 'currency': 'USD', 'min'...",PROMOTED BY VENUE,"[{'primary': True, 'segment': {'id': 'KZFzniwn..."
3,G5dIZpELxbwy-,Jackopierce,[Jackopierce],Austin City Limits Live at The Moody Theater,Austin,TX,30.26536691,-97.7471222,2021-04-12T01:00:00Z,America/Chicago,False,[],[],2021-03-24T19:00:00Z,"[{'type': 'standard', 'currency': 'USD', 'min'...",PROMOTED BY VENUE,"[{'primary': True, 'segment': {'id': 'KZFzniwn..."
4,G5dIZpeE09UbK,Jackopierce,[Jackopierce],Austin City Limits Live at The Moody Theater,Austin,TX,30.26536691,-97.7471222,2021-04-12T01:00:00Z,America/Chicago,False,[2020-01-16T16:00:00Z],[2020-01-17T04:00:00Z],2020-01-17T16:00:00Z,"[{'type': 'standard', 'currency': 'USD', 'min'...",PROMOTED BY VENUE,"[{'primary': True, 'segment': {'id': 'KZFzniwn..."


## Initial Data Preparation

**Events with same name but different dates and markets have the same TM_id. I am dropping duplicates for this initial analysis.**<br>
**Also dropping all parking related ticket sales**

In [8]:
#Drop duplicates
tm_df.drop_duplicates(subset='TM_id',inplace=True)

#Remove parking passes
tm_df = tm_df.loc[~tm_df['TM_venue'].str.lower().str.contains('parking')]
tm_df = tm_df.loc[~tm_df['TM_name'].str.lower().str.contains('parking')]

**Expanding out the price column, genre columnm, and ID column**

In [10]:
#Explode price column
tm_df = explode(tm_df,col='TM_FV_prices',index_col='TM_id')
#Unnest genre info and clean up data in exploded columns
tm_df = explode(tm_df,'TM_genre','TM_id')
for col in ['genre','segment','subGenre']:
    tm_df[col] = tm_df[col].map(lambda x: dict(x).get('name',{}))
tm_df.head()

,TM_id,TM_name,TM_artist,TM_venue,TM_venue_city,TM_venue_state,TM_venue _lat,TM_venue_long,TM_date,TM_timezone,...,max,min,type_x,primary,segment,genre,subGenre,type_y,subType,family
0,G5dIZp0wKy-SF,Justin Furstenfeld of Blue October (21+ Event)...,[Justin Furstenfeld],Whitewater Amphitheater,New Braunfels,TX,29.862013,-98.156969,2021-04-24T01:00:00Z,America/Chicago,...,102.00,43.00,standard,True,Music,Rock,Pop,"{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False
1,G5dIZpbCwKv7f,Jamestown Revival (21+ Event) (Reduced Capacit...,[Jamestown Revival],Whitewater Amphitheater,New Braunfels,TX,29.862013,-98.156969,2021-04-04T01:00:00Z,America/Chicago,...,45.00,23.00,standard,True,Music,Rock,Alternative Rock,"{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False
2,G5dIZ4URqF9t2,The Monkees,[The Monkees],Austin City Limits Live at The Moody Theater,Austin,TX,30.26536691,-97.7471222,2021-04-03T01:00:00Z,America/Chicago,...,80.75,40.75,standard,True,Music,Rock,Pop,"{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False
3,G5dIZpELxbwy-,Jackopierce,[Jackopierce],Austin City Limits Live at The Moody Theater,Austin,TX,30.26536691,-97.7471222,2021-04-12T01:00:00Z,America/Chicago,...,30.00,25.00,standard,True,Music,Rock,Pop,"{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False
4,G5dIZpeE09UbK,Jackopierce,[Jackopierce],Austin City Limits Live at The Moody Theater,Austin,TX,30.26536691,-97.7471222,2021-04-12T01:00:00Z,America/Chicago,...,25.00,25.00,standard,True,Music,Rock,Pop,"{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False


In [11]:
tm_df.shape

(105, 27)

In [14]:
tm_df.sample(20)

,TM_id,TM_name,TM_artist,TM_venue,TM_venue_city,TM_venue_state,TM_venue _lat,TM_venue_long,TM_date,TM_timezone,...,max,min,type_x,primary,segment,genre,subGenre,type_y,subType,family
75,G5dIZpkEOs0DN,Celtic Thunder - Ireland,[Celtic Thunder],Majestic Theatre San Antonio,San Antonio,TX,29.4264514,-98.4903863,2021-11-14T01:00:00Z,America/Chicago,...,69.50,39.50,standard,True,Music,World,World,"{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False
57,G5dIZ4gBvkwAd,Buddy Guy,[Buddy Guy],Austin City Limits Live at The Moody Theater,Austin,TX,30.26536691,-97.7471222,2022-03-21T01:00:00Z,America/Chicago,...,79.50,49.50,standard,True,Music,Blues,Blues,"{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False
35,G5dIZ4UUfFwoR,Keith Sweat,"[Keith Sweat, SWV, All-4-One]",H-E-B Center at Cedar Park,Cedar Park,TX,30.5393195,-97.82448,2021-05-01T00:30:00Z,America/Chicago,...,75.00,35.00,standard,True,Music,R&B,R&B,"{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False
12,G5dIZp7Mm0Mtn,PRIMUS - A Tribute to Kings,"[Primus, Wolfmother, The Sword]",Austin City Limits Live at The Moody Theater,Austin,TX,30.26536691,-97.7471222,2021-06-13T00:00:00Z,America/Chicago,...,79.50,47.00,standard,True,Music,Rock,Alternative Rock,"{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False
3,G5dIZ4Ml6IKRh,Alanis Morissette w/special guest Garbage & al...,"[Alanis Morissette, Garbage, Liz Phair]",Germania Insurance Amphitheater,Austin,TX,30.13310378,-97.64039993,2021-08-13T00:00:00Z,America/Chicago,...,166.00,65.50,standard,True,Music,Rock,Pop,"{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False
3,G5dIZpELxbwy-,Jackopierce,[Jackopierce],Austin City Limits Live at The Moody Theater,Austin,TX,30.26536691,-97.7471222,2021-04-12T01:00:00Z,America/Chicago,...,30.00,25.00,standard,True,Music,Rock,Pop,"{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False
54,G5dIZpA3cP7gV,Judas Priest: 50 Heavy Metal Years,"[Judas Priest, Sabaton]",Freeman Coliseum,San Antonio,TX,29.42597,-98.4428439,2021-10-13T01:00:00Z,America/Chicago,...,129.50,49.50,standard,True,Music,Metal,Nu-Metal,"{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False
64,G5dIZpA9IcEPy,Smokey Robinson,[Smokey Robinson],H-E-B Center at Cedar Park,Cedar Park,TX,30.5393195,-97.82448,2021-06-27T01:00:00Z,America/Chicago,...,150.00,55.00,standard,True,Music,R&B,R&B,"{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False
27,G5dIZp7R4Y4tt,Widespread Panic 3-Day Pass,[Widespread Panic],Austin City Limits Live at The Moody Theater,Austin,TX,30.26536691,-97.7471222,{},America/Chicago,...,378.00,207.00,standard,True,Music,Rock,Alternative Rock,"{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False
10,G5dIZp7QVaMxP,Jason Isbell and the 400 Unit,"[Jason Isbell and the 400 Unit, Lucinda Williams]",Austin City Limits Live at The Moody Theater,Austin,TX,30.26536691,-97.7471222,2021-08-08T01:00:00Z,America/Chicago,...,89.50,59.50,standard,True,Music,Rock,Pop,"{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}","{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}",False


In [15]:
tm_df.columns

Index([                'TM_id',               'TM_name',
                   'TM_artist',              'TM_venue',
               'TM_venue_city',        'TM_venue_state',
               'TM_venue _lat',         'TM_venue_long',
                     'TM_date',           'TM_timezone',
       'TM_span_multiple_days', 'TM_presale_date_start',
         'TM_presale_date_end',    'TM_sale_date_start',
                 'TM_promoter',                       0,
                    'currency',                   'max',
                         'min',                'type_x',
                     'primary',               'segment',
                       'genre',              'subGenre',
                      'type_y',               'subType',
                      'family'],
      dtype='object')

In [16]:
#Conert data to .csv
tm_df.to_csv('tm_events_040121_050121')